# iBioFoundry - Golden Gate Assembly - Echo
author: Camillo Moschner | version: 2.0 | date: 16.08.2022 | license = 

What this code does:
- Performs liquid handling of parts, utilising the Echo's power to combine diverse H<sub>2</sub>O-like liquids with speed and accuracy; but it does not assemble the full reaction as the reaction buffer and enzymes are better manually added after part transfer.

# 1- Metadata

In [1]:
metadata = {
    'protocolName': '# Automated Design and/or Assembly of Golden Gate Reactions',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Programme for the automated design and/or assembly of Golden Gate reactions',
    'apiLevel': '2.10',
    'Date': '25.09.2022',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

## Assembly Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [2]:
# establish whether you are performing a simulation (e.g. at the design stage) or are executing the script on the OT-2
script_mode = 'simulation' # 'simulation' or 'exection'
# input data
experiment_name = 'PREEx'
design_file_directory = 'input1a_design_file_factorial_JUMP_PREEx.csv' # 'design_file_defined_CIDAR_test1.csv'
reagent_definitions_dir = 'input1b_reagent_definitions_MoClo_CIDAR_LVL1_BsaI.csv'

# choose total volumes (typically 1/2 of total GGA reaction volume)
total_part_volume = 1.2 # units: µl
total_reaction_volume = 2.4 # units: µl

# choose naming convention for newly created plasmids:
first_part_ID = ('pCM',301)
# choose plate formats:
ark_plate_ID = '220925_ark_384_sTUFPlibrary'
moladj_parts_format = '384PP_AQ_BP2'

assembly_plate_ID = '220817_assembly_sTUFPlibrary'
assembly_plate_format = '384_PCR_plate Framestar - 4titude' #'opentrons_96_aluminumblock_biorad_wellplate_200ul' #' # opentrons_96_aluminumblock_biorad_wellplate_200ul
assembly_plate_column_usage = 'all' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tubes w/ snapcaps
assembly_plate_empty_margin = 1 # chose how many rows & columns from the edge of the plate you want to leave empty, e.g. to avoid evaporation effects

# 2- Import Statements

In [3]:
from itertools import chain, product
from datetime import date
from IPython.display import display, clear_output, Audio, display
from copy import deepcopy
import pandas as pd
import numpy as np
import time
import math
import os

from iBioFoundry_helper import *
from opentrons import protocol_api, execute, simulate

# 3- Function Definitions

---
# 4- Deck assignments

In [4]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,3))
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list), axis=0) )

In [5]:
deck_slot_df

,0
0,2
1,1


## i. Ark plate info integration

In [6]:
sample_definitionslayout_dir = f'0_ark_plate_info_{experiment_name}{os.path.sep}1_ark_plate_info.csv'
sample_definitions_df = pd.read_csv(sample_definitionslayout_dir)
sample_definitions_df['position_tuple'] = sample_definitions_df['moladj_parts_positions'].apply(lambda name: (name[0],int(name[1:])))

# create new moladj_parts_plate_obj and 
moladj_parts_plate_obj = Plate(identify_plate(moladj_parts_format, 'plate_format'))
assembly_plate_obj = Plate(identify_plate(assembly_plate_format, 'plate_format'),
                           only_columns=assembly_plate_column_usage,
                           empty_margin=assembly_plate_empty_margin)

# update the new moladj_parts_plate_obj with the precise location of the prepped, molarity-adjusted parts
for part_info in [sample_definitions_df.iloc[row,:] for row in range(len(sample_definitions_df))]:
    moladj_parts_plate_obj.layout.loc[part_info['position_tuple']]=part_info['name']
sample_definitions_df

,name,concentration (nM),total volume (ul),moladj_parts_positions,ark_plate_format,prep_date,position_tuple,sample_vol_before_2023-06-04
0,P_T7(BBF10K_003378)_15nM,15.0,46.34,A1,96,23/05/2023,"(A, 1)",110.02
1,P_T7_lacO(BBF10K_003379)_15nM,15.0,42.98,B1,96,23/05/2023,"(B, 1)",106.66
2,RBS_BT1_BCD2(BBF10K_003384)_15nM,15.0,91.08,C1,96,23/05/2023,"(C, 1)",122.92
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,15.0,137.86,D1,96,23/05/2023,"(D, 1)",169.70
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,15.0,66.86,E1,96,23/05/2023,"(E, 1)",98.70
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,15.0,97.98,F1,96,23/05/2023,"(F, 1)",129.82
6,Ntag1_R5(BBF10K_003399)_30nM,30.0,67.92,G1,96,23/05/2023,"(G, 1)",67.92
7,Ntag2_TEVcut(BBF10K_003418)_30nM,30.0,67.62,H1,96,23/05/2023,"(H, 1)",67.62
8,Ntag_His_TEVcut(BBF10K_003407_15nM,15.0,57.28,A2,96,23/05/2023,"(A, 2)",120.96
9,CDS_Eco31I(BBF10K_003292)_15nM,15.0,115.38,B2,96,23/05/2023,"(B, 2)",131.38


---
# 5- LH Step Calculations 
## Design file reading

In [7]:
design_info_int = pd.read_csv(design_file_directory,nrows=0).columns.tolist()
design_info = list(chain.from_iterable([x.split(':') for x in design_info_int]))
design_info = {design_info[i]: design_info[i+1] for i in range(0, len(design_info), 2)}
if (design_info['design type'] == 'factorial') or (design_info['design type'] == 'defined'):
    _=[print(f"{key} : {design_info[key]}") for key in design_info.keys()]
else:
    print(f"WARNING: Design type not specified!")

design type : factorial
GG standard : uLoop
assembly level : LVL1
LHS : Opentrons_OT2
date : 10.05.2023
Unnamed :  5


In [8]:
design_file_df = pd.read_csv(design_file_directory,header=1) 
# create complete list of DNA parts required:
complete_DNA_parts_list_required = list(chain.from_iterable([ sorted(set(notna_data(design_file_df[name]))) for name in design_file_df.columns ]))
display(design_file_df)
#display(complete_DNA_parts_list_required)
print(f"\n -> {len(complete_DNA_parts_list_required)} parts needed \n")

,Module A | Part 1: Promoter,Module A | Part 2: RBS,Module A | Part 3 & 4: Tag1and2,Module A | Part 5: CDS,Module A | Part 6: Terminator,Module A | Part 7: LVL1 Destination Vector
0,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_Eco31I(BBF10K_003292)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM
1,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_EcoRI(BBF10K_003281)_15nM,NaN,NaN
2,NaN,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,NaN,CDS_NotI(BBF10K_003300)_15nM,NaN,NaN
3,NaN,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,NaN,CDS_PstI(BBF10K_003283)_15nM,NaN,NaN
4,NaN,NaN,NaN,CDS_SapI(BBF10K_003295)_15nM,NaN,NaN
5,NaN,NaN,NaN,CDS_XbaI(BBF10K_003298)_15nM,NaN,NaN



 -> 16 parts needed 



## Create combinations

In [9]:
combinations_list = list(product(*[notna_data(design_file_df[part]) for part in design_file_df.columns])) 

if design_info['design type'] == 'factorial':
    combinations_df = pd.DataFrame(combinations_list,columns=design_file_df.columns.tolist())
elif design_info['design type'] == 'defined':
    combinations_df = deepcopy(design_file_df)
combinations_no = len(combinations_df)
part_per_assembly_no = len(design_file_df.columns)
# identify most diverse part
comb_description = combinations_df.describe()
part_uniques = pd.DataFrame(comb_description.loc['unique',:])
most_diverse_part = part_uniques.loc[part_uniques['unique'] == part_uniques['unique'].max()]

if int(part_uniques.var()) == 0:
    print(f"All parts are equi-present & therefore there is no \'most_diverse_part\'. Since one has to be defined for further steps though the first part will have been assigned this variable \n")
else:
    print(f"The current design would create - {combinations_no} - different assembly combinations/new plasmids.\n\'{most_diverse_part.index[0]}\' is the most diverse part.\n")
comb_description

The current design would create - 96 - different assembly combinations/new plasmids.
'Module A | Part 5: CDS' is the most diverse part.



,Module A | Part 1: Promoter,Module A | Part 2: RBS,Module A | Part 3 & 4: Tag1and2,Module A | Part 5: CDS,Module A | Part 6: Terminator,Module A | Part 7: LVL1 Destination Vector
count,96,96,96,96,96,96
unique,2,4,2,6,1,1
top,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_Eco31I(BBF10K_003292)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM
freq,48,24,48,16,96,96


---

## Assembly plate check
Check that there is enough space in the assembly plate for all the calculated assemblies.

In [10]:
Plate(384,empty_margin=1).layout

,2,3,4,5,6,7,8,9,10,11,...,14,15,16,17,18,19,20,21,22,23
B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
K,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
usable_destination_plate_pos_list = [index+str(column) for column in assembly_plate_obj.layout.columns for index in assembly_plate_obj.layout.index ]
no_usable_destination_plate_positions = len(assembly_plate_obj.layout.columns)*len(assembly_plate_obj.layout.index)
if combinations_no <= no_usable_destination_plate_positions:
    print(f"✅ All clear: You are trying to assemble {combinations_no} constructs and have {len(usable_destination_plate_pos_list)} positions available on the assembly plate.")
elif (combinations_no > no_usable_destination_plate_positions) and (combinations_no <= 96):
    print(f"""\nWARNING: You are trying to assemble {combinations_no} constructs but only have {len(usable_destination_plate_pos_list)} positions available on the assembly plate!
    -> You have to use the full assembly_plate! e.g. with a BioRad 96-well PCR plate.\n""")
elif combinations_no > 96:
    print(f"""\nWARNING: You are trying to assemble {combinations_no} constructs but only have {len(usable_destination_plate_pos_list)} positions available on the assembly plate!
    -> You have to use the full assembly_plate in multiple iterations! e.g. use 2x BioRad 96-well PCR plates.\n""")

✅ All clear: You are trying to assemble 96 constructs and have 308 positions available on the assembly plate.


## Create Summary DF

In [12]:
# assign each assembly an final assembly_tube, assembly_ID and assembly_description
combinations_df['assembly_ID'] = create_part_IDs_list(first_part_ID,combinations_no)
extend_fill_plate_df_with_list(assembly_plate_obj.layout, combinations_df.assembly_ID.to_list(), fill_start_position='A1', inplace=True, fill_first='columns')
combinations_df['assembly_tube'] = list(assembly_plate_obj.occupied_wells.keys())#usable_destination_plate_pos_list[:combinations_no]
transfer_vol = total_part_volume/part_per_assembly_no *1000 # unit conversion from µl to nl
try:
    combinations_df['assembly_description'] = combinations_df['assembly_description'] # in case you run this cell multiple times (otherwise would expand combinations_df['assembly_description'] on every call)
except:
    assembly_descriptions = ['|'.join(combinations_df.loc[:,design_file_df.columns].loc[row_idx,:].tolist()) for row_idx in np.arange(combinations_no)]
combinations_df['assembly_description'] = assembly_descriptions
combinations_df['assembly_date'] = [str(date.today()) for i in np.arange(combinations_no)]
# move important info to the fron of the spreadsheet
for column_name in ['assembly_description', 'assembly_ID']:
    first_column = combinations_df.pop(column_name)
    combinations_df.insert(0, column_name, first_column)

In [13]:
#combinations_df

## - Visual Deck Inspection

In [14]:
print("__Part Plate/Source Plate__")
display(moladj_parts_plate_obj.layout)
print("\n__Assembly Plate/Destination Plate__")
display(assembly_plate_obj.layout)

__Part Plate/Source Plate__


,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
A,P_T7(BBF10K_003378)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,DV_pTi_7.5nM,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B,P_T7_lacO(BBF10K_003379)_15nM,CDS_Eco31I(BBF10K_003292)_15nM,DV_pTi_5nM,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,RBS_BT1_BCD2(BBF10K_003384)_15nM,CDS_EcoRI(BBF10K_003281)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,CDS_NotI(BBF10K_003300)_15nM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,CDS_PstI(BBF10K_003283)_15nM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
F,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,CDS_SapI(BBF10K_003295)_15nM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G,Ntag1_R5(BBF10K_003399)_30nM,CDS_XbaI(BBF10K_003298)_15nM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H,Ntag2_TEVcut(BBF10K_003418)_30nM,T_TZ(BBF10K_003477)_15nM,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



__Assembly Plate/Destination Plate__


,2,3,4,5,6,7,8,9,10,11,...,14,15,16,17,18,19,20,21,22,23
B,pCM301,pCM315,pCM329,pCM343,pCM357,pCM371,pCM385,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C,pCM302,pCM316,pCM330,pCM344,pCM358,pCM372,pCM386,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,pCM303,pCM317,pCM331,pCM345,pCM359,pCM373,pCM387,0,0,0,...,0,0,0,0,0,0,0,0,0,0
E,pCM304,pCM318,pCM332,pCM346,pCM360,pCM374,pCM388,0,0,0,...,0,0,0,0,0,0,0,0,0,0
F,pCM305,pCM319,pCM333,pCM347,pCM361,pCM375,pCM389,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G,pCM306,pCM320,pCM334,pCM348,pCM362,pCM376,pCM390,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H,pCM307,pCM321,pCM335,pCM349,pCM363,pCM377,pCM391,0,0,0,...,0,0,0,0,0,0,0,0,0,0
I,pCM308,pCM322,pCM336,pCM350,pCM364,pCM378,pCM392,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J,pCM309,pCM323,pCM337,pCM351,pCM365,pCM379,pCM393,0,0,0,...,0,0,0,0,0,0,0,0,0,0
K,pCM310,pCM324,pCM338,pCM352,pCM366,pCM380,pCM394,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create Echo Picklist

In [15]:
summary_lst = []
for assembly in [combinations_df.iloc[row_no,:] for row_no in range(len(combinations_df))]:
    parts_used = assembly.assembly_description.split('|')
    for current_sample in parts_used:
        template_dict = {
            'assembly_ID' : assembly.assembly_ID,
            'Sample Name' : current_sample,
            'Transfer Volume' : transfer_vol, # units: nl
            'Source Plate Name' : ark_plate_ID,
            'Source Plate Type' : moladj_parts_format,
            'Source Well' : find_df_coordinates(moladj_parts_plate_obj.layout, current_sample),
            'Destination Plate Name' : assembly_plate_ID,
            'Destination Plate Type' : assembly_plate_format,
            'Destination Well' : assembly.assembly_tube
        }
        summary_lst.append(template_dict)
echo_picklist_df = pd.DataFrame(summary_lst)

In [16]:
echo_picklist_df

,assembly_ID,Sample Name,Transfer Volume,Source Plate Name,Source Plate Type,Source Well,Destination Plate Name,Destination Plate Type,Destination Well
0,pCM301,P_T7(BBF10K_003378)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,A1,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,B2
1,pCM301,RBS_BT1_BCD2(BBF10K_003384)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,C1,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,B2
2,pCM301,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,C3,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,B2
3,pCM301,CDS_Eco31I(BBF10K_003292)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,B2,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,B2
4,pCM301,T_TZ(BBF10K_003477)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,H2,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,B2
...,...,...,...,...,...,...,...,...,...
571,pCM396,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,F1,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,M8
572,pCM396,Ntag_His_TEVcut(BBF10K_003407_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,A2,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,M8
573,pCM396,CDS_XbaI(BBF10K_003298)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,G2,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,M8
574,pCM396,T_TZ(BBF10K_003477)_15nM,200.0,220925_ark_384_sTUFPlibrary,384PP_AQ_BP2,H2,220817_assembly_sTUFPlibrary,384_PCR_plate Framestar - 4titude,M8


## Part usage summary

In [17]:
DNA_part_counts = []
_=[DNA_part_counts.append(combinations_df[part_name].value_counts()) for part_name in design_file_df.columns.tolist()]
part_names, part_counts = [], []
[part_names.append(DNA_part_counts[idx].index.tolist()) for idx in np.arange(len(DNA_part_counts))], [part_counts.append(DNA_part_counts[idx].tolist()) for idx in np.arange(len(DNA_part_counts))]
part_names, part_counts = list(chain.from_iterable(part_names)), list(chain.from_iterable(part_counts))
part_types = [find_df_coordinates(design_file_df,part_name,tuple_result=True)[-1][-1] if type(find_df_coordinates(design_file_df,part_name,tuple_result=True))!=tuple else find_df_coordinates(design_file_df,part_name,tuple_result=True)[-1] for part_name in part_names]
# convert part usage statistics into pandas.DataFrame, calculate insurance volumes for each and show each's position in the parts_rack
part_usage_summary_df = pd.DataFrame([part_types,part_names,part_counts]).T
part_usage_summary_df.rename(columns={0: 'part_type', 1: 'part_name', 2: 'usages'},inplace=True)
part_usage_summary_df['insurance_vol'] = part_usage_summary_df['usages']*transfer_vol/1_000+ identify_plate(moladj_parts_format, 'dead_vol')/1000 # 20 µl dead volume (to insure against calibration issues & well_bottom_clearance problems)
part_usage_summary_df['position'] = [find_df_coordinates(moladj_parts_plate_obj.layout,part_name,tuple_result=True)[0]+str(find_df_coordinates(moladj_parts_plate_obj.layout,part_name,tuple_result=True)[1]).zfill(2) for part_name in part_usage_summary_df['part_name']]
part_usage_summary_df.sort_values(by=['part_type','position'],inplace=True)
part_usage_summary_df.reset_index(drop=True,inplace=True)
# check availability of the volume required for all parts in the molarity-adjusted parts plate
availability_checkup = []
leftover_vol = []
for required_part_info in [part_usage_summary_df.iloc[row_no,:] for row_no in range(len(part_usage_summary_df))]:
    current_part_name = required_part_info['part_name']
    current_insurance_vol = required_part_info['insurance_vol']
    current_part_in_plate_info = sample_definitions_df.loc[sample_definitions_df.name == current_part_name ]
    leftover_in_plate_after_assembly = current_part_in_plate_info['total volume (ul)'].iloc[0] - current_insurance_vol
    if leftover_in_plate_after_assembly > 0:
        availability_checkup.append('ok')
    else: 
        print(f"WARNING: {current_part_name} has not enough molaritised miniprep available!")
        availability_checkup.append('WARNING')
    leftover_vol.append(round(leftover_in_plate_after_assembly,2))
part_usage_summary_df['availability'] = availability_checkup
part_usage_summary_df['leftover'] = leftover_vol
part_usage_summary_df['leftover_check'] = ['ok' if vol > identify_plate(moladj_parts_format, 'dead_vol')/1_000 else 'WARNING' for vol in leftover_vol] # defines the dead vol -> requires checking
part_usage_summary_df

,part_type,part_name,usages,insurance_vol,position,availability,leftover,leftover_check
0,Module A | Part 1: Promoter,P_T7(BBF10K_003378)_15nM,48,9.615,A01,ok,36.73,ok
1,Module A | Part 1: Promoter,P_T7_lacO(BBF10K_003379)_15nM,48,9.615,B01,ok,33.36,ok
2,Module A | Part 2: RBS,RBS_BT1_BCD2(BBF10K_003384)_15nM,24,4.815,C01,ok,86.26,ok
3,Module A | Part 2: RBS,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,24,4.815,D01,ok,133.05,ok
4,Module A | Part 2: RBS,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,24,4.815,E01,ok,62.04,ok
5,Module A | Part 2: RBS,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,24,4.815,F01,ok,93.16,ok
6,Module A | Part 3 & 4: Tag1and2,Ntag_His_TEVcut(BBF10K_003407_15nM,48,9.615,A02,ok,47.67,ok
7,Module A | Part 3 & 4: Tag1and2,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,48,9.615,C03,ok,46.70,ok
8,Module A | Part 5: CDS,CDS_Eco31I(BBF10K_003292)_15nM,16,3.215,B02,ok,112.16,ok
9,Module A | Part 5: CDS,CDS_EcoRI(BBF10K_003281)_15nM,16,3.215,C02,ok,124.54,ok


## Reagent preparation instructions

In [18]:
assembly_surplus = 10 # %
reagent_definitions_df = pd.read_csv(reagent_definitions_dir, index_col=0)
work_vol_given = float(reagent_definitions_df.columns[0].split('_')[-1].split('ul')[0])
downscale_factor = work_vol_given/total_reaction_volume
 

reagent_definitions_df[f"1x_work_vol_{int(total_reaction_volume)}ul"] = reagent_definitions_df['1x_work_vol_10ul']/downscale_factor
downscaled_reagent_vol = reagent_definitions_df[f"1x_work_vol_{int(total_reaction_volume)}ul"].sum()
one_x_water = (total_reaction_volume-downscaled_reagent_vol)-total_part_volume
reagent_definitions_df[f"1x_work_vol_{int(total_reaction_volume)}ul"].loc['water1']=one_x_water


prep_col_name, dead_vol_comp_factor = f"{combinations_no}x_prep_vol_{int(total_reaction_volume)}ul", 10/downscaled_reagent_vol
reagent_definitions_df[f"1x_prep_vol_{int(total_reaction_volume)}ul"] = round(reagent_definitions_df[f"1x_work_vol_{int(total_reaction_volume)}ul"]*(assembly_surplus/100+1),2)


In [19]:
reagent_definitions_df[prep_col_name] = round(reagent_definitions_df[f"1x_prep_vol_{int(total_reaction_volume)}ul"]*combinations_no,2)
reagent_definitions_df[prep_col_name] = reagent_definitions_df[prep_col_name]+reagent_definitions_df[f"1x_work_vol_{int(total_reaction_volume)}ul"]*dead_vol_comp_factor
print(f"\nA master mix of {round(reagent_definitions_df[prep_col_name].sum(),2)} µl has to be prepared.\n");display(reagent_definitions_df);


A master mix of 145.76 µl has to be prepared.



,1x_work_vol_10ul,1x_work_vol_2ul,1x_prep_vol_2ul,96x_prep_vol_2ul
water1,NaN,0.60,0.66,73.36
T4_DNAligase_buffer,1.0,0.24,0.26,28.96
BSA,0.5,0.12,0.13,14.48
T4_DNAligase,0.5,0.12,0.13,14.48
BsaI-HFv2,0.5,0.12,0.13,14.48


## Summary Spreadsheets

In [20]:
save_spreadsheet_answer = input("Do you want to save the summary spreadsheets? (yes / no)\n answer:")
if save_spreadsheet_answer == 'yes':
    try:
        os.makedirs('1_GGA')
    except:
        pass
    # save prepped equimolar sample information
    combinations_df.to_csv('1_GGA/assemblies_summary.csv',index=False)
    reagent_definitions_df.to_csv('1_GGA/reagent_tracker_summary.csv',index=False)
    echo_picklist_df.to_csv(f"1_GGA/{date.today()}_echo_picklist.csv",index=False)
    print(f"The GGA summary spreadsheets have been saved to \'1_GGA{os.path.sep}\'!")
else:
    print(f"\nYou have not saved the GGA summary spreadsheets!\n")

Do you want to save the summary spreadsheets? (yes / no)
 answer: yes


The GGA summary spreadsheets have been saved to '1_GGA/'!


---
# 6- Execution

- Prep ark_plate & reagent MM, and go to Echo
- Use Echo_picklist file to assemble parts into assembly_plate
- manually add reagent MM (multi-channel pipette is highly recommended)

# Part Plate Update

In [21]:
# integrate volume tracking
part_usage_dict = round(echo_picklist_df['Sample Name'].value_counts()*(total_part_volume/len(design_file_df.columns)),2)
part_usage_dict = part_usage_dict.to_dict()
sample_definitions_df[f"sample_vol_used_{str(date.today())}"] = sample_definitions_df.name.apply(lambda x: part_usage_dict.get(x,0.0))
update_moladj_parts_plate_answer = input(f"\nDo you want to update the part_plate volume information with todays part_usage?\nThis will override the saved part_plate spreadsheet.\n\n ----> yes / no ? \n\n")
if update_moladj_parts_plate_answer == 'yes':
    sample_definitions_df[f"sample_vol_before_{str(date.today())}"] = sample_definitions_df['total volume (ul)']
    sample_definitions_df['total volume (ul)'] = round(sample_definitions_df['total volume (ul)'] - sample_definitions_df[f"sample_vol_used_{str(date.today())}"],2)
    #sample_definitions_df.drop(f'sample_vol_used_{str(date.today())}',axis=1,inplace=True)
    sample_definitions_df.to_csv(sample_definitionslayout_dir,index=False)
    print(f"\nExcellent! Now you can continue running the rest of the protocol.")
else:
    print(f"\nEnsure you keep an accurate record of when you remove liquid from your ark plate.\n")


Do you want to update the part_plate volume information with todays part_usage?
This will override the saved part_plate spreadsheet.

 ----> yes / no ? 

 yes



Excellent! Now you can continue running the rest of the protocol.


In [22]:
sample_definitions_df#['total volume (ul)'] #- sample_definitions_df[f"sample_vol_used_{str(date.today())}"]

,name,concentration (nM),total volume (ul),moladj_parts_positions,ark_plate_format,prep_date,position_tuple,sample_vol_before_2023-06-04,sample_vol_used_2023-06-04
0,P_T7(BBF10K_003378)_15nM,15.0,36.74,A1,96,23/05/2023,"(A, 1)",46.34,9.6
1,P_T7_lacO(BBF10K_003379)_15nM,15.0,33.38,B1,96,23/05/2023,"(B, 1)",42.98,9.6
2,RBS_BT1_BCD2(BBF10K_003384)_15nM,15.0,86.28,C1,96,23/05/2023,"(C, 1)",91.08,4.8
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,15.0,133.06,D1,96,23/05/2023,"(D, 1)",137.86,4.8
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,15.0,62.06,E1,96,23/05/2023,"(E, 1)",66.86,4.8
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,15.0,93.18,F1,96,23/05/2023,"(F, 1)",97.98,4.8
6,Ntag1_R5(BBF10K_003399)_30nM,30.0,67.92,G1,96,23/05/2023,"(G, 1)",67.92,0.0
7,Ntag2_TEVcut(BBF10K_003418)_30nM,30.0,67.62,H1,96,23/05/2023,"(H, 1)",67.62,0.0
8,Ntag_His_TEVcut(BBF10K_003407_15nM,15.0,47.68,A2,96,23/05/2023,"(A, 2)",57.28,9.6
9,CDS_Eco31I(BBF10K_003292)_15nM,15.0,112.18,B2,96,23/05/2023,"(B, 2)",115.38,3.2


# Shutdown
  * N/A